### Alice möchte Bob 10000 satoshis senden welche Bob anschließend wieder ausgibt. Bob gibt Alice dabei verschiedene Möglichkeiten wie die Satoshis für ihn gelocked werden.

Die ersten Möglichkeit ist das Versenden der Satoshis an einen öffentlichen Schlüssel welcher durch einen von Bob kontrollierten privaten Schlüssel erzeugt wurde. Dieser Fall wird **P2PK-Transaktion** genannt.

In [1]:
from io import BytesIO

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2pk_script
from src.ecdsa import PrivateKey

**1.** In der ersten Transaktion werden von Alice die 10000 Satoshis an den von Bob kontrollierten öffentlichen Schlüssel versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen. 

In [2]:
alice_private_key = 109238526296081169617111611626239112771658584030722025481308193087698209243021
alice_public_key = PrivateKey(alice_private_key).get_public_key()
alice_previous_tx_id_to_spent = '181463bd39030c73ea0ede82abbcc38cdb8021f8649c975eca58cbf4213ccb00'
alice_previous_tx_index_to_spent = 1
alice_previous_script_pub_key_to_spent = '76a914a3a88b764d10f463584ee465887e046d114b611a88ac'
alice_amount_to_spent = 10000
bob_public_key = PrivateKey(70391367530871559720919750735257129489246656383889830217409950893950252925507).get_public_key()

**1.1** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's öffentlichen Schlüssel zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: <Bob's public Key> + CHECKSIG generieren.

In [3]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2pk_script(bob_public_key.sec_format())
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2PK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice unsignierte P2PK Lockingtransaktion: 010000000100cb3c21f4cb58ca5e979c64f82180db8cc3bcab82de0eea730c0339bd6314180100000000ffffffff011027000000000000232103982f610d344db57d441e594d4d4f5001fe1369081a42f8350b49be562e94dfb4acffffffff


**1.1.1** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [4]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, alice_private_key, alice_script_sig)

print(f"Alice signierte P2PK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice signierte P2PK Lockingtransaktion: 010000000100cb3c21f4cb58ca5e979c64f82180db8cc3bcab82de0eea730c0339bd631418010000006b48304502210084885e00d4a71500e5ffd761e122fd8b54b20d936699f22ac978dc506987056c0220619d0ef5a8c93fe28b04c4a44a1149c13d4aa480c42cdf6b50af3c542cf9dd25012103dc3ca2b6b61aa84cfc62fbf9189d66a44a895d4d172b9df36f1628073f2da899ffffffff011027000000000000232103982f610d344db57d441e594d4d4f5001fe1369081a42f8350b49be562e94dfb4acffffffff


**2.** Nun möchte Bob die an seinen öffentlichen Schlüssel gesendeten Satoshis weiter versenden. Dazu muss Bob im Inputfeld (in dem auf den eben von Alice erzeugten Output referenziert wird), zum Beweis das er  berechtigt ist aen Output auszugeben lediglich eine Signatur mit Bob's privatem Schlüssel stehen. Der Einfachheit halber sendet Bob die neue Transaktion an den selben public Key wie die Alice. **Bitte macht das NIEMALS NIEMALS NIEMALS für eine echte Transaktion auf der Mainchain!!!!!**
Damit Bob das notwendige Transaktionsformular ausfüllen kann benötigt er folgende Informationen.

In [5]:
bob_private_key = 70391367530871559720919750735257129489246656383889830217409950893950252925507
bob_previous_tx_id_to_spent = '8f237d79cdb261d67cdf4dfe1e2ff7d776cf05ceefb01b6d273f29152a30e48c'
bob_previous_tx_index_to_spent = 0
bob_previous_script_pub_key_to_spent = "2103982f610d344db57d441e594d4d4f5001fe1369081a42f8350b49be562e94dfb4ac"#alice_script_pubkey.serialize_script().hex()
bob_amount_to_spent = 9000

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [6]:
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script())

script_pubkey = p2pk_script(alice_public_key.sec_format())
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Bob unsignierte P2PK Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")


Bob unsignierte P2PK Spendingtransaktion: 01000000018ce4302a15293f276d1bb0efce05cf76d7f72f1efe4ddf7cd661b2cd797d238f0000000000ffffffff012823000000000000232103dc3ca2b6b61aa84cfc62fbf9189d66a44a895d4d172b9df36f1628073f2da899acffffffff


**2.2** Nun kann Bob diese Transaktion mit seinem privaten Schlüssle signieren und versenden.

In [7]:
bob_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(bob_previous_script_pub_key_to_spent)//2)[2:]}{bob_previous_script_pub_key_to_spent}")))
bob_spending_transaction.sign_transaction(0, bob_private_key, bob_script_sig)

print(f"Bob signierte P2PK Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

Bob signierte P2PK Spendingtransaktion: 01000000018ce4302a15293f276d1bb0efce05cf76d7f72f1efe4ddf7cd661b2cd797d238f0000000049483045022100bc209892a8217e7940a2f8ce0fd7e615d94f174a8e6c0b3cfd06c53f6ffb1dac0220611400acfedc0255e3870aa2ad1501527db275ca016f9e9c025702c2c0625aba01ffffffff012823000000000000232103dc3ca2b6b61aa84cfc62fbf9189d66a44a895d4d172b9df36f1628073f2da899acffffffff


now let's try this on bitcoin regtest
1. bitcoind -regtest -daemon -fallbackfee=0.0000025
2. bcr createwallet alice
3. bcr createwallet bob
4. bob's wallet loaded: bcr getnewaddress legacy bcr -rpcwallet=bob getnewaddress "" legacy result: muSAzMRK75CzLUCCFjy6TLgVr5vQ1ufkxF
5. now we need bob's private key associated with this address bcr -rpcwallet=bob dumpprivkey muSAzMRK75CzLUCCFjy6TLgVr5vQ1ufkxF, result is in wif-format: cSoDYWXxTwTbnWBpTsgWiaJbD4ZTLpJ51nHppZRFKtCK418ERJEo
6. now we need to convert bob's private key in wif to int format result: 70391367530871559720919750735257129489246656383889830217409950893950252925507
7. now we load alice wallet and generate 101 blocks to alice -> so alice now has some btc in her wallet
8. for a better overview we will generate a new address for alice where we will send 11000 sats -> this will be the input tx for our p2pk-locking tx
9. now we need the private key for the address we sent the 11000 satoshis to: cVgAfaNWLeHR4fGgERYwVmYUwZ51D3d3BzFkxszxYdDG9vw99hLn -> 109238526296081169617111611626239112771658584030722025481308193087698209243021
10. with this we can now generate the unsigned raw p2pk-locking transaction: 010000000100cb3c21f4cb58ca5e979c64f82180db8cc3bcab82de0eea730c0339bd6314180100000000ffffffff011027000000000000232103982f610d344db57d441e594d4d4f5001fe1369081a42f8350b49be562e94dfb4acffffffff
11. now we sign this tx with alice private_key: 010000000100cb3c21f4cb58ca5e979c64f82180db8cc3bcab82de0eea730c0339bd631418010000006b48304502210084885e00d4a71500e5ffd761e122fd8b54b20d936699f22ac978dc506987056c0220619d0ef5a8c93fe28b04c4a44a1149c13d4aa480c42cdf6b50af3c542cf9dd25012103dc3ca2b6b61aa84cfc62fbf9189d66a44a895d4d172b9df36f1628073f2da899ffffffff011027000000000000232103982f610d344db57d441e594d4d4f5001fe1369081a42f8350b49be562e94dfb4acffffffff
12. now we can send/broadcast this tx: 8f237d79cdb261d67cdf4dfe1e2ff7d776cf05ceefb01b6d273f29152a30e48c
13. after mining a new block bob now controls with his private_key a new unspent transaction output
14. to spend this tx bob can now create a new unsigned spending p2pk-tx: 01000000018ce4302a15293f276d1bb0efce05cf76d7f72f1efe4ddf7cd661b2cd797d238f0000000000ffffffff012823000000000000232103982f610d344db57d441e594d4d4f5001fe1369081a42f8350b49be562e94dfb4acffffffff
15. with his private_key bob can sign this tx: 01000000018ce4302a15293f276d1bb0efce05cf76d7f72f1efe4ddf7cd661b2cd797d238f0000000049483045022100bc209892a8217e7940a2f8ce0fd7e615d94f174a8e6c0b3cfd06c53f6ffb1dac0220611400acfedc0255e3870aa2ad1501527db275ca016f9e9c025702c2c0625aba01ffffffff012823000000000000232103dc3ca2b6b61aa84cfc62fbf9189d66a44a895d4d172b9df36f1628073f2da899acffffffff